In [37]:
import pandas as pd
import numpy as np

In [39]:
sample = pd.read_csv('./small_sample.csv')

In [3]:
df = pd.concat([sample,sample,sample,sample,sample,sample],axis = 0)

In [4]:
def devide(t,h):
    if h == 0:
        return np.nan
    return t/h

In [20]:
%%timeit
iterrows_devide = []
for index,row in df.iterrows():
    Time = row['Time']
    HelpfulnessNumerator = row['HelpfulnessNumerator']
    iterrows_devide.append(devide(Time,HelpfulnessNumerator))

5min 12s ± 31.1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit
df.apply(lambda row: devide(row['Time'],row['HelpfulnessNumerator']),axis = 1)

1min 25s ± 4.88 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit
itertuples_devide = []
for _ in df.itertuples():
    Time = getattr(_,'Time')
    HelpfulnessNumerator = getattr(_,'HelpfulnessNumerator')
    itertuples_devide.append(devide(Time,HelpfulnessNumerator))

4 s ± 238 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit df['Time'] /df['HelpfulnessNumerator']

10.2 ms ± 655 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit list(map(devide,df['Time'],df['HelpfulnessNumerator']))

861 ms ± 22.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit np.vectorize(devide)(df['Time'],df['HelpfulnessNumerator'])

636 ms ± 24.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
def modified_devide(t,h):
    return t/h

In [30]:
%timeit np.where(df['HelpfulnessNumerator'] == 0 , 0 , modified_devide(df['Time'],df['HelpfulnessNumerator']))

21.4 ms ± 200 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%timeit modified_devide(df['Time'],df['HelpfulnessNumerator'])

8.75 ms ± 114 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
%timeit np.where(df['HelpfulnessNumerator'] == 0 , 0 , df['Time']/df['HelpfulnessNumerator'])

21.6 ms ± 329 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%timeit df['Time']/df['HelpfulnessNumerator']

8.83 ms ± 140 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
from numba import njit

@njit()
def divide(t, h):
    res = np.empty(t.shape)
    for i in range(len(t)):
        if h[i] != 0:
            res[i] = t[i] / h[i]
        else:
            res[i] = 0
    return res

%timeit divide(df['Time'].values, df['HelpfulnessNumerator'].values)

9.66 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%load_ext fortranmagic

In [35]:
%%fortran
       subroutine fortran_array(x, y, z)
            real, intent(in) :: x(:), y(:)
            real, intent(out) :: z(size(x))
            ! using vector operations
            z(:) = x(:) / y(:)
       end subroutine

In [36]:
%timeit fortran_array(df['Time'],df['HelpfulnessNumerator'])

11.7 ms ± 910 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
my_function(df['Time'],df['HelpfulnessNumerator'])

array([1.3038624e+09,           inf, 1.2190176e+09, ..., 6.6489120e+08,
       1.3315968e+09,           inf], dtype=float32)

In [15]:
%%fortran -vvv --f90flags='-fopenmp' --extra='-lgomp'
       subroutine openmp_fortran(x, y, z)
            use omp_lib
            real, intent(in) :: x(:), y(:)
            real, intent(out) :: z(size(x))
            z(:) = x(:) / y(:)
       end subroutine

Running...
   /Users/mahdi.rasoulinezhad/code/my/bin/python3 -m numpy.f2py --f90flags='-fopenmp' -lgomp -m _fortran_magic_5f1065350196776a8da595cc4ae935cc -c /Users/mahdi.rasoulinezhad/.ipython/fortran/_fortran_magic_5f1065350196776a8da595cc4ae935cc.f90
running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "_fortran_magic_5f1065350196776a8da595cc4ae935cc" sources
f2py options: []
f2py:> /var/folders/js/yzxj9gss565_mv0q0bcjg5pc77vzp5/T/tmp8akd3n0t/src.macosx-10.15-x86_64-3.7/_fortran_magic_5f1065350196776a8da595cc4ae935ccmodule.c
creating /var/folders/js/yzxj9gss565_mv0q0bcjg5pc77vzp5/T/tmp8akd3n0t/src.macosx-10.15-x86_64-3.7
Reading fortran codes...
	Reading file '/Users/mahdi.rasoulinezhad/.ipython/fortran/_fortran_magic_5f1065350196776a8da595cc4ae935cc.f90' (format:fix)
Post-

In file included from /var/folders/js/yzxj9gss565_mv0q0bcjg5pc77vzp5/T/tmp8akd3n0t/src.macosx-10.15-x86_64-3.7/fortranobject.c:2:
In file included from /var/folders/js/yzxj9gss565_mv0q0bcjg5pc77vzp5/T/tmp8akd3n0t/src.macosx-10.15-x86_64-3.7/fortranobject.h:13:
In file included from /Users/mahdi.rasoulinezhad/code/my/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /Users/mahdi.rasoulinezhad/code/my/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/mahdi.rasoulinezhad/code/my/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1832:
/Users/mahdi.rasoulinezhad/code/my/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
1 warning generated.
In file included fr


Ok. The following fortran objects are ready to use: openmp_fortran


In [16]:
%timeit openmp_fortran(df['Time'],df['HelpfulnessNumerator'])

12 ms ± 535 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%fortran

subroutine f2(x, y, z)
    real, intent(in) :: x,y
    real, intent(out) :: z
    
    z = x/y
    IF ( 0 .EQ. Y ) THEN 
         Z = 0
    END IF
    
    
end subroutine f2

In [18]:
f2(1,0)

0.0

In [19]:
%timeit np.vectorize(f2)(df['Time'],df['HelpfulnessNumerator'])

/Users/mahdi.rasoulinezhad/code/my/lib/python3.7/site-packages/numpy/lib/function_base.py:2167: RuntimeWarning: divide by zero encountered in ? (vectorized)
  outputs = ufunc(*inputs)


674 ms ± 20.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
np.where(df)